In [1]:
import os
import sys
from datetime import datetime, timedelta, timezone
import pandas as pd

module_path = os.path.abspath(os.path.join("..", "src"))
sys.path.insert(0, str(module_path))
from eduhub_queries import EdHubDB


In [2]:
# Part 1

edhub = EdHubDB()
edhub.build_collection()

Created collection: users
Created collection: courses
Created collection: lessons
Created collection: assignments
Created collection: enrollments
Created collection: submissions


In [3]:
# Part 2
edhub.seed_database()

Seeded 20 documents into 'users' collection.
Seeded 8 documents into 'courses' collection.
Seeded 15 documents into 'enrollments' collection.
Seeded 15 documents into 'lessons' collection.
Seeded 10 documents into 'assignments' collection.
Seeded 12 documents into 'submissions' collection.


## Part 3 - Basic CRUD Operations
###  Create Operations

In [4]:
# insert user

user_id = "u100"
edhub.insert_student(edhub.helper.make_user(user_id))

# verify user is inserted
edhub.users_col.find_one({"userId": user_id})

{'_id': ObjectId('684df7c35300822bf0c48fc7'),
 'userId': 'u100',
 'email': 'qthompson@example.net',
 'firstName': 'Sean',
 'lastName': 'Rodriguez',
 'role': 'student',
 'dateJoined': datetime.datetime(2025, 4, 29, 4, 4, 26, 67000),
 'profile': {'bio': 'Drive special according television enjoy establish nothing.',
  'avatar': 'https://placekitten.com/863/262',
  'skills': ['JavaScript', 'Machine Learning', 'MongoDB']},
 'is_active': True}

In [5]:
# insert course
course_id = "c100"
user_id = "u2" # role is instructor
edhub.insert_course(edhub.helper.make_course(course_id=course_id, instructor_id=user_id))

# verify course is inserted
edhub.courses_col.find_one({"courseId": course_id})

{'_id': ObjectId('684df7c35300822bf0c48fc8'),
 'courseId': 'c100',
 'title': 'Blood view seat first of middle mind.',
 'description': 'Street maintain thought quite. Result away sort person season subject five. Staff contain evening everybody lay.',
 'instructorId': 'u2',
 'category': 'Data Engineering',
 'level': 'intermediate',
 'duration': 60,
 'price': 6132,
 'tags': ['Python', 'MongoDB'],
 'createdAt': datetime.datetime(2023, 4, 29, 9, 46, 7, 430000),
 'updatedAt': datetime.datetime(2024, 11, 12, 7, 8, 2, 708000),
 'isPublished': False}

In [6]:
# Enroll a student into a course
user_id = "u100" # existing user that's a student
course_id = "c100" # existing course

inserted_id = edhub.register_student(student_id=user_id, course_id=course_id)

# verify enrollment is created
print(inserted_id)
edhub.enrollments_col.find_one({"_id": inserted_id})

684df7c35300822bf0c48fc9


{'_id': ObjectId('684df7c35300822bf0c48fc9'),
 'enrollmentId': 'e16',
 'studentId': 'u100',
 'courseId': 'c100',
 'enrollmentDate': datetime.datetime(1991, 1, 1, 12, 59, 25, 818000),
 'progress': 0.0,
 'completed': False,
 'certificateIssued': False}

In [7]:
# Add a new lesson to an existing course
course_id = "c100" # existing course
lesson_id = "l100"

edhub.insert_lesson(edhub.helper.make_lesson(lesson_id=lesson_id, course_id=course_id))

# verify lesson is created
edhub.lessons_col.find_one({"lessonId": lesson_id})

{'_id': ObjectId('684df7c45300822bf0c48fca'),
 'lessonId': 'l100',
 'courseId': 'c100',
 'title': 'Five market agree account speech cup himself.',
 'content': 'Best return bad. Bill question note grow many someone. Industry radio kid very near material position. Loss down light reduce its authority discussion. Tough clear north example someone.',
 'order': 82,
 'resources': ['intro.pdf'],
 'duration': 30,
 'createdAt': datetime.datetime(2023, 3, 17, 6, 21, 22, 32000),
 'updatedAt': datetime.datetime(2025, 2, 24, 19, 35, 49, 287000)}

### Read Operations

In [8]:
# Find all active students
students = edhub.get_active_students()
if students:
    df_students = pd.DataFrame(students)
    display(df_students[["userId", "firstName", "lastName", "email", "role", "isActive"]])
else:
    print("No active students found.")

,userId,firstName,lastName,email,role,isActive
0,u1,Alice,Johnson,alice.student@example.com,student,True
1,u3,Carol,Lee,carol.student@example.com,student,True
2,u5,Eve,Davis,eve.student@example.com,student,True
3,u7,Grace,Wilson,grace.student@example.com,student,True
4,u9,Ivy,Taylor,ivy.student@example.com,student,True
5,u11,Kate,Thomas,kate.student@example.com,student,True
6,u13,Mia,White,mia.student@example.com,student,True
7,u15,Oliver,Martin,oliver.student@example.com,student,True
8,u17,Quinn,Lewis,quinn.student@example.com,student,True
9,u19,Sam,Hall,sam.student@example.com,student,True


In [51]:
# Retrieve course details with instructor information
courses = edhub.get_course_details()
if courses:
    import pandas as pd

    # Flatten instructor info for display
    for course in courses:
        instructor = course.get("instructor", {})
        course["instructorName"] = f"{instructor.get('firstName', '')} {instructor.get('lastName', '')}"
        course["instructorEmail"] = instructor.get("email", "")
        course["instructorBio"] = instructor.get("profile", {}).get("bio", "")
        course["instructorSkills"] = ", ".join(instructor.get("profile", {}).get("skills", []))

    df_courses = pd.DataFrame(courses)
    display(df_courses[[
        "courseId", "title", "category", "level", "price", "rating",
        "instructorName", "instructorEmail", "instructorBio", "instructorSkills"
    ]])
else:
    print("No course details found.")

,courseId,title,category,level,price,rating,instructorName,instructorEmail,instructorBio,instructorSkills
0,c1,Introduction to JavaScript,Programming,beginner,0,3.0,Bob Smith,bob.instructor@example.com,Full-stack instructor.,"Node.js, MongoDB"
1,c2,Advanced Python,Programming,advanced,50,4.0,Henry Moore,henry.instructor@example.com,AI/ML instructor.,"TensorFlow, PyTorch"
2,c3,Web Design Fundamentals,Design,beginner,20,4.0,Nina Harris,nina.instructor@example.com,UI/UX designer.,"Figma, Sketch"
3,c4,Database Management,Database,intermediate,30,5.0,Jack Anderson,jack.instructor@example.com,Database expert.,"MongoDB, SQL"
4,c5,Cloud Computing Basics,Cloud,beginner,40,2.0,Paul Clark,paul.instructor@example.com,Cloud architect.,"AWS, GCP"
5,c6,Mobile App Development,Mobile,intermediate,60,4.0,Rachel Walker,rachel.instructor@example.com,Mobile app developer.,"Flutter, Swift"
6,c7,DevOps Essentials,DevOps,intermediate,35,4.0,Frank Miller,frank.instructor@example.com,DevOps engineer.,"Docker, Kubernetes"
7,c8,Cybersecurity Fundamentals,Security,beginner,25,5.0,Leo Jackson,leo.instructor@example.com,Cybersecurity instructor.,"Security, Linux"
8,c100,Blood view seat first of middle mind.,Data Engineering,intermediate,6132,NaN,Bob Smith,bob.instructor@example.com,Full-stack instructor.,"Node.js, MongoDB"


In [10]:
# Get all courses in a specific category
courses = edhub.get_courses_by_category("Programming")
if courses:
    df_courses = pd.DataFrame(courses)
    display(df_courses[["courseId", "title", "category", "level", "price", "rating"]])
else:
    print("No courses found in this category.")

,courseId,title,category,level,price,rating
0,c1,Introduction to JavaScript,Programming,beginner,0,3
1,c2,Advanced Python,Programming,advanced,50,4


In [54]:
# Find students enrolled in a particular course (include profile fields)
course_id = "c1"
students = edhub.get_student_enrolled_to_course(course_id=course_id)
if students:
    import pandas as pd
    # Flatten profile fields for each student
    for student in students:
        profile = student.get("profile", {})
        student["bio"] = profile.get("bio", "")
        student["avatar"] = profile.get("avatar", "")
        student["skills"] = ", ".join(profile.get("skills", []))
    df_students = pd.DataFrame(students)
    display(df_students[[
        "userId", "firstName", "lastName", "email", "role", "dateJoined",
        "bio", "avatar", "skills", "isActive"
    ]])
else:
    print("No students found for this course.")

,userId,firstName,lastName,email,role,dateJoined,bio,avatar,skills,isActive
0,u1,Alice,Johnson,alice.student@example.com,student,2025-04-10 09:00:00,Software engineer,alice.png,"Ruby, Go",False
1,u17,Quinn,Lewis,quinn.student@example.com,student,2024-06-11 09:00:00,Interested in DevOps.,quinn.jpg,"Docker, CI/CD",True


In [12]:
# Search courses by title (case-insensitive, partial match)
partial_title = "py"

edhub.search_courses_by_title(partial_title)

[{'_id': ObjectId('684df7c35300822bf0c48f8c'),
  'courseId': 'c2',
  'title': 'Advanced Python',
  'description': 'Deep dive into Python.',
  'instructorId': 'u8',
  'category': 'Programming',
  'level': 'advanced',
  'duration': 20,
  'price': 50,
  'tags': ['Python', 'Advanced', 'Web'],
  'createdAt': datetime.datetime(2024, 2, 1, 9, 0),
  'updatedAt': datetime.datetime(2024, 2, 10, 9, 0),
  'isPublished': True,
  'rating': 4}]

### Update Operations

In [13]:
# Update a user’s profile information
user_id = "u1"

user = edhub.users_col.find_one({"userId": user_id})

print("Original: ", user["profile"])

profile = {
    "bio": "Software engineer",
    "avatar": "alice.png",
    "skills": ["Ruby", "Go"]
}

edhub.modify_profile(user_id=user_id, updates=profile)

user = edhub.users_col.find_one({"userId": user_id})

print("Updated: ", user["profile"])

Original:  {'bio': 'Aspiring developer.', 'avatar': 'alice.jpg', 'skills': ['JavaScript', 'HTML']}
Updated:  {'bio': 'Software engineer', 'avatar': 'alice.png', 'skills': ['Ruby', 'Go']}


In [14]:
# Mark a course as published
course_id = "c1"

course = edhub.courses_col.find_one({"courseId": course_id})
print("Original: ", course["isPublished"])

edhub.publish_course(course_id=course_id)
course = edhub.courses_col.find_one({"courseId": course_id})
print("Updated: ", course["isPublished"])


Original:  False
Updated:  True


In [15]:
# Update assignment grade

submission_id = "s1"
grade = 40
feedback = "Good work"

submission = edhub.submissions_col.find_one({"submissionId": submission_id})
print("Original: ", submission["grade"])

edhub.update_assignment_grade(submission_id=submission_id, grade=grade, feedback=feedback)

submission = edhub.submissions_col.find_one({"submissionId": submission_id})
print("Updated: ", submission["grade"])

Original:  95
Updated:  40


In [16]:
# Add tags to an existing course
course_id = "c1"

course = edhub.courses_col.find_one({"courseId": course_id})
print("Original: ", course["tags"])

edhub.add_tags_to_course(course_id=course_id, tags=["Nodejs"])

course = edhub.courses_col.find_one({"courseId": course_id})
print("Updated: ", course["tags"])


Original:  ['JavaScript', 'Web']
Updated:  ['JavaScript', 'Web', 'Nodejs']


### Delete Operations

In [17]:
# Remove a user (soft delete by setting isActive to false)
user_id = "u1"

user = edhub.users_col.find_one({"userId": user_id})
print("Original: ", user["isActive"])

edhub.deactivate_user(user_id=user_id)

user = edhub.users_col.find_one({"userId": user_id})
print("Updated: ", user["isActive"])


Original:  True
Updated:  False
Updated:  False
Updated:  False


In [18]:
# Delete an enrollment

enrollment_id = "e15"

enrollment = edhub.enrollments_col.find_one({"enrollmentId": enrollment_id})
print("Original: ", enrollment)

edhub.delete_enrollment(enrollment_id=enrollment_id)

enrollment = edhub.enrollments_col.find_one({"enrollmentId": enrollment_id})
print("Updated: ", enrollment)

Original:  {'_id': ObjectId('684df7c35300822bf0c48fa1'), 'enrollmentId': 'e15', 'studentId': 'u9', 'courseId': 'c7', 'enrollmentDate': datetime.datetime(2024, 6, 18, 9, 0), 'progress': 1.0, 'completed': True, 'certificateIssued': True}
Updated:  None


In [19]:
# Remove a lesson from a course
lesson_id = "l1"
course_id = "c1"

lesson = edhub.lessons_col.find_one({"lessonId": lesson_id})
print("Original: ", lesson)

edhub.remove_lesson_from_course(lesson_id=lesson_id, course_id=course_id)

lesson = edhub.lessons_col.find_one({"lessonId": lesson_id})
print("Updated: ", lesson)


Original:  {'_id': ObjectId('684df7c35300822bf0c48fa2'), 'lessonId': 'l1', 'courseId': 'c1', 'title': 'JS Introduction', 'content': 'Welcome to JavaScript.', 'order': 1, 'resources': ['intro.pdf'], 'duration': 30, 'createdAt': datetime.datetime(2024, 1, 1, 9, 0), 'updatedAt': datetime.datetime(2024, 1, 2, 9, 0)}
Updated:  {'_id': ObjectId('684df7c35300822bf0c48fa2'), 'lessonId': 'l1', 'courseId': '', 'title': 'JS Introduction', 'content': 'Welcome to JavaScript.', 'order': 1, 'resources': ['intro.pdf'], 'duration': 30, 'createdAt': datetime.datetime(2024, 1, 1, 9, 0), 'updatedAt': datetime.datetime(2024, 1, 2, 9, 0)}


## Part 4: Advanced Queries and Aggregation (25 points)

### Complex Queries

In [50]:
# Find courses with price between $50 and $200
min_price = 50
max_price = 200

courses = edhub.courses_by_price(min_price, max_price)
if courses:
    import pandas as pd
    df = pd.DataFrame(courses)
    display(df[["courseId", "title", "price", "category", "level"]])
else:
    print("No courses found in this price range.")

,courseId,title,price,category,level
0,c2,Advanced Python,50,Programming,advanced
1,c6,Mobile App Development,60,Mobile,intermediate


In [49]:
# Get users who joined in the last 6 months
recent_users = edhub.recent_signups(months=6)
if recent_users:
    import pandas as pd

    # Normalize the profile dictionary for each user
    for user in recent_users:
        profile = user.get("profile", {})
        user["bio"] = profile.get("bio", "")
        user["avatar"] = profile.get("avatar", "")
        user["skills"] = ", ".join(profile.get("skills", []))
        # Handle both isActive and is_active keys
        user["isActive"] = user.get("isActive", user.get("is_active", ""))

    df_recent = pd.DataFrame(recent_users)
    display(df_recent[[
        "userId", "firstName", "lastName", "email", "role", "dateJoined",
        "bio", "avatar", "skills", "isActive"
    ]])
else:
    print("No recent signups found.")

,userId,firstName,lastName,email,role,dateJoined,bio,avatar,skills,isActive
0,u1,Alice,Johnson,alice.student@example.com,student,2025-04-10 09:00:00.000,Software engineer,alice.png,"Ruby, Go",False
1,u2,Bob,Smith,bob.instructor@example.com,instructor,2025-02-15 10:00:00.000,Full-stack instructor.,bob.jpg,"Node.js, MongoDB",True
2,u100,Sean,Rodriguez,qthompson@example.net,student,2025-04-29 04:04:26.067,Drive special according television enjoy estab...,https://placekitten.com/863/262,"JavaScript, Machine Learning, MongoDB",True


In [22]:
# Find courses that have specific tags using $in operator
keywords = ["Web", "Python"]
courses_with_tags = edhub.courses_with_keyword(keywords)
if courses_with_tags:
    df_tags = pd.DataFrame(courses_with_tags)
    display(df_tags[["courseId", "title", "tags", "category"]])
else:
    print("No courses found with these tags.")

,courseId,title,tags,category
0,c1,Introduction to JavaScript,"[JavaScript, Web, Nodejs]",Programming
1,c2,Advanced Python,"[Python, Advanced, Web]",Programming
2,c4,Database Management,"[Database, SQL, MongoDB, Web]",Database
3,c100,Blood view seat first of middle mind.,"[Python, MongoDB]",Data Engineering


In [46]:
# Retrieve assignments with due dates in the next week
assignment_id = "a1"

# update assignment_id "a1" due date to three days from now
new_due_date = datetime.now(timezone.utc) + timedelta(days=3)
edhub.assignments_col.update_one(
    {"assignmentId": assignment_id},
    {"$set": {"dueDate": new_due_date}}
)

# get assignments due in the next week
assignments = edhub.upcoming_assignment_due_date(upcoming_week=1)
if assignments:
    import pandas as pd
    df_assignments = pd.DataFrame(assignments)
    display(df_assignments)
else:
    print("No assignments due in the next week.")

,_id,assignmentId,courseId,lessonId,title,instructions,dueDate,maxScore,createdAt,updatedAt
0,684df7c35300822bf0c48fb1,a1,c1,l1,JS Quiz 1,Complete the quiz.,2025-06-17 22:37:58.016,100,2024-01-01 09:00:00,2024-01-02 09:00:00


#### Course Enrollment Statistics:

In [24]:
# Count total enrollments per course
enrollments = edhub.enrollment_metrics()
if enrollments:
    df_enrollments = pd.DataFrame(enrollments)
    display(df_enrollments[["courseId", "courseTitle", "totalEnrollments"]])
else:
    print("No enrollment metrics found.")

,courseId,courseTitle,totalEnrollments
0,c5,Cloud Computing Basics,2
1,c3,Web Design Fundamentals,2
2,c7,DevOps Essentials,1
3,c8,Cybersecurity Fundamentals,1
4,c100,Blood view seat first of middle mind.,1
5,c4,Database Management,2
6,c6,Mobile App Development,2
7,c1,Introduction to JavaScript,2
8,c2,Advanced Python,2


In [25]:
# Calculate average course rating
edhub.average_course_rating()

{'averageRating': 3.875, 'count': 9}

In [45]:
# Group by course category
categories = edhub.group_course_by_category()
if categories:
    import pandas as pd
    df_categories = pd.DataFrame(categories)
    display(df_categories[["category", "courses", "totalCourses"]])
else:
    print("No course category data found.")

,category,courses,totalCourses
0,Programming,"[Introduction to JavaScript, Advanced Python]",2
1,Cloud,[Cloud Computing Basics],1
2,DevOps,[DevOps Essentials],1
3,Data Engineering,[Blood view seat first of middle mind.],1
4,Database,[Database Management],1
5,Security,[Cybersecurity Fundamentals],1
6,Mobile,[Mobile App Development],1
7,Design,[Web Design Fundamentals],1


#### Student Performance Analysis

In [27]:
# Average grade per student
grades = edhub.average_grade_per_student()
if grades:
    df_grades = pd.DataFrame(grades)
    display(df_grades[["studentId", "studentName", "averageGrade", "submissions"]])
else:
    print("No grade data found.")

,studentId,studentName,averageGrade,submissions
0,u3,Carol Lee,84.0,2
1,u1,Alice Johnson,40.0,1
2,u13,Mia White,93.0,1
3,u9,Ivy Taylor,90.0,1
4,u15,Oliver Martin,89.0,1
5,u7,Grace Wilson,85.0,1
6,u11,Kate Thomas,87.0,1
7,u17,Quinn Lewis,91.0,1
8,u5,Eve Davis,88.5,2
9,u19,Sam Hall,94.0,1


In [41]:
# Completion rate by course
completion = edhub.course_completion_rate()
if completion:
    import pandas as pd
    df_completion = pd.DataFrame(completion)
    display(df_completion[["courseId", "completionRate", "totalEnrolled"]])
else:
    print("No course completion data found.")

,courseId,completionRate,totalEnrolled
0,c2,0.5,2
1,c6,0.0,2
2,c5,0.0,2
3,c7,0.0,1
4,c8,0.0,1
5,c4,0.0,2
6,c100,0.0,1
7,c3,0.0,2
8,c1,0.0,2


In [38]:
# Top-performing students by average grade
top_students = edhub.top_performing_students()
if top_students:
    import pandas as pd
    df_top_students = pd.DataFrame(top_students)
    display(df_top_students[["studentId", "studentName", "averageGrade", "submissions"]])
else:
    print("No top-performing student data found.")

,studentId,studentName,averageGrade,submissions
0,u19,Sam Hall,94.0,1
1,u13,Mia White,93.0,1
2,u17,Quinn Lewis,91.0,1
3,u9,Ivy Taylor,90.0,1
4,u15,Oliver Martin,89.0,1


### Instructor Analytics

In [37]:
# Total students taught by each instructor
instructor_stats = edhub.total_student_by_each_instructor()
if instructor_stats:
    import pandas as pd
    df_instructors = pd.DataFrame(instructor_stats)
    display(df_instructors[["instructorId", "instructorName", "totalStudents", "coursesTaught"]])
else:
    print("No instructor student data found.")

,instructorId,instructorName,totalStudents,coursesTaught
0,u6,Frank Miller,1,[c7]
1,u8,Henry Moore,2,[c2]
2,u14,Nina Harris,2,[c3]
3,u12,Leo Jackson,1,[c8]
4,u2,Bob Smith,3,"[c1, c100]"
5,u10,Jack Anderson,2,[c4]
6,u16,Paul Clark,2,[c5]
7,u18,Rachel Walker,2,[c6]


In [31]:
# Average course rating per instructor
ratings = edhub.average_course_rating_per_instructor()
if ratings:
    import pandas as pd
    df_ratings = pd.DataFrame(ratings)
    display(df_ratings[["instructorId", "instructorName", "averageRating"]])
else:
    print("No instructor rating data found.")

,instructorId,instructorName,averageRating
0,u14,Nina Harris,4.0
1,u10,Jack Anderson,5.0
2,u6,Frank Miller,4.0
3,u16,Paul Clark,2.0
4,u2,Bob Smith,3.0
5,u18,Rachel Walker,4.0
6,u8,Henry Moore,4.0
7,u12,Leo Jackson,5.0


In [32]:
# Revenue generated per instructor
revenue = edhub.revenue_per_instructor()
if revenue:
    import pandas as pd
    df_revenue = pd.DataFrame(revenue)
    display(df_revenue[["instructorId", "instructorName", "revenue"]])
else:
    print("No revenue data found.")

,instructorId,instructorName,revenue
0,u18,Rachel Walker,120
1,u14,Nina Harris,40
2,u8,Henry Moore,100
3,u12,Leo Jackson,25
4,u2,Bob Smith,6132
5,u10,Jack Anderson,60
6,u16,Paul Clark,80
7,u6,Frank Miller,35


#### Advanced Analytics - 3

In [33]:
# Monthly enrollment trends 
trends = edhub.montly_enrollment_trend()
if trends:
    import pandas as pd
    df_trends = pd.DataFrame(trends)
    display(df_trends[["year", "month", "totalEnrollments"]])
else:
    print("No enrollment trend data found.")

,year,month,totalEnrollments
0,1991,1,1
1,2024,1,1
2,2024,2,1
3,2024,3,1
4,2024,4,1
5,2024,5,1
6,2024,6,9


In [34]:
# Most popular course categories
categories = edhub.most_popular_course_categories()
if categories:
    import pandas as pd
    df_categories = pd.DataFrame(categories)
    display(df_categories[["category", "totalCourses"]])
else:
    print("No course category data found.")

,category,totalCourses
0,Programming,2
1,DevOps,1
2,Data Engineering,1
3,Cloud,1
4,Design,1


In [35]:
# Student engagement metrics (e.g., submissions per student)
engagement = edhub.student_engagement_metrics()
if engagement:
    import pandas as pd
    df_engagement = pd.DataFrame(engagement)
    display(df_engagement[["studentId", "studentName", "totalSubmissions", "averageGrade"]])
else:
    print("No engagement data found.")

,studentId,studentName,totalSubmissions,averageGrade
0,u3,Carol Lee,2,84.0
1,u13,Mia White,1,93.0
2,u1,Alice Johnson,1,40.0
3,u9,Ivy Taylor,1,90.0
4,u15,Oliver Martin,1,89.0
5,u7,Grace Wilson,1,85.0
6,u11,Kate Thomas,1,87.0
7,u17,Quinn Lewis,1,91.0
8,u5,Eve Davis,2,88.5
9,u19,Sam Hall,1,94.0


## Part 5: Indexing and Performance

In [36]:
import time

# SLOW QUERY: Case-insensitive regex search on lastName (no index)
query = {"lastName": {"$regex": "son", "$options": "i"}}

# Drop index if it exists to simulate base query
try:
    edhub.users_col.drop_index("lastName_1")
except Exception:
    pass  # Ignore if index doesn't exist

# Run query without index
start = time.time()
explain_no_index = edhub.users_col.find(query).explain()
end = time.time()
print("Time without index: {:.4f} seconds".format(end - start))
print("Winning plan (no index):", explain_no_index["queryPlanner"]["winningPlan"]["stage"])

# Create index on lastName
edhub.users_col.create_index([("lastName", 1)])

# Run query with index
start = time.time()
explain_with_index = edhub.users_col.find(query).explain()
end = time.time()
print("Time with index: {:.4f} seconds".format(end - start))
print("Winning plan (with index):", explain_with_index["queryPlanner"]["winningPlan"]["stage"])

# Use projection to return only userId and lastName
start = time.time()
explain_projection = edhub.users_col.find(query, {"userId": 1, "lastName": 1, "_id": 0}).explain()
end = time.time()
print("Time with index and projection: {:.4f} seconds".format(end - start))
print("Winning plan (projection):", explain_projection["queryPlanner"]["winningPlan"]["stage"])

Time without index: 0.0031 seconds
Winning plan (no index): COLLSCAN
Time with index: 0.0048 seconds
Winning plan (with index): FETCH
Time with index and projection: 0.0019 seconds
Winning plan (projection): PROJECTION_SIMPLE
